# ***Growth Rate (YOY Rate)***

1.  <a href="#country">Country YOY</a>
2.  <a href="#specific_c">Country YOY (Totals for specific months)</a>
3.  <a href="#product">Products YOY</a>
4.  <a href="#specific_p_h2">Products YOY (Totals for specific months) </a>


### ***Note:***
Since the Profit Margin is 7% of the Net Sales for all products, then its Growth rate will be the same as SalesYOY (No need to calculate its Growth Rate)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
server = 'HEND\SQLEXPRESS'
db_name = 'NORTHWND'
driver = 'ODBC Driver 17 for SQL Server'
connect_db= f'mssql://@{server}/{db_name}?driver={driver}'
engine = create_engine(connect_db)
con_db= engine.connect()

<a id='country'></a>
# 1- Country YOY

In [6]:
# Loading Data
country_report=pd.read_sql_query('select * from [dbo].[Country_Transactions]',con_db)

In [8]:
country_report.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit
0,Norway,1997-Nor,1997,700.00,49.00
1,Denmark,1998-Den,1998,4516.09,316.14
2,Venezuela,1997-Ven,1997,26404.92,1848.36
3,Germany,1997-Ger,1997,117320.18,8212.48
4,France,1997-Fra,1997,45263.38,3168.46


In [10]:
country_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        62 non-null     object 
 1   Country_ID     62 non-null     object 
 2   Year           62 non-null     int64  
 3   Annual_Sales   62 non-null     float64
 4   Annual_Profit  62 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 2.6+ KB


In [12]:
#Sorting Data:
country_report.sort_values(by= ['Country','Year'], inplace= True,ignore_index=True)
country_report.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit
0,Argentina,1997-Arg,1997,1816.60,127.17
1,Argentina,1998-Arg,1998,6302.50,441.18
2,Austria,1996-Aus,1996,25601.34,1792.10
3,Austria,1997-Aus,1997,57401.86,4018.15
4,Austria,1998-Aus,1998,45000.65,3150.08


In [14]:
# Calculating YOY for Sales :
country_report['Sales_YOY'] = country_report.groupby('Country').Annual_Sales.pct_change(periods=1)
country_report.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997-Arg,1997,1816.60,127.17,NaN
1,Argentina,1998-Arg,1998,6302.50,441.18,2.469393
2,Austria,1996-Aus,1996,25601.34,1792.10,NaN
3,Austria,1997-Aus,1997,57401.86,4018.15,1.242143
4,Austria,1998-Aus,1998,45000.65,3150.08,-0.216042


In [16]:
country_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        62 non-null     object 
 1   Country_ID     62 non-null     object 
 2   Year           62 non-null     int64  
 3   Annual_Sales   62 non-null     float64
 4   Annual_Profit  62 non-null     float64
 5   Sales_YOY      41 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.0+ KB


In [18]:
country_report

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997-Arg,1997,1816.60,127.17,NaN
1,Argentina,1998-Arg,1998,6302.50,441.18,2.469393
2,Austria,1996-Aus,1996,25601.34,1792.10,NaN
3,Austria,1997-Aus,1997,57401.86,4018.15,1.242143
4,Austria,1998-Aus,1998,45000.65,3150.08,-0.216042
...,...,...,...,...,...,...
57,USA,1997-USA,1997,114845.27,8039.28,2.013862
58,USA,1998-USA,1998,92633.67,6484.43,-0.193405
59,Venezuela,1996-Ven,1996,9738.10,681.67,NaN
60,Venezuela,1997-Ven,1997,26404.92,1848.36,1.711506


In [20]:
# Replace Null Values with 0:
country_report.fillna(0, inplace=True)
country_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        62 non-null     object 
 1   Country_ID     62 non-null     object 
 2   Year           62 non-null     int64  
 3   Annual_Sales   62 non-null     float64
 4   Annual_Profit  62 non-null     float64
 5   Sales_YOY      62 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.0+ KB


#### Reshaping the Data

In [50]:
country_report.query('Year == 1996').shape

(20, 6)

In [26]:
country_report.query('Year == 1997').shape

(21, 6)

In [28]:
country_report.query('Year == 1998').shape

(21, 6)

In [30]:
# Reshaping the Data:
new_df= country_report.query('Year == 1996').iloc[:,[0,3,4]].rename(columns={'Annual_Sales':'Sales_96','Annual_Profit':'Profit_96'})

new_df=new_df.merge(country_report.query('Year == 1997').iloc[:,[0,3,4,5]], how = 'right').rename(columns={'Annual_Sales':'Sales_97',
                                                                                                           'Annual_Profit':'Profit_97',
                                                                                                            'Sales_YOY':'YOY_97'})

new_df= new_df.merge(country_report.query('Year == 1998').iloc[:,[0,3,4,5]], how = 'right').rename(columns={'Annual_Sales':'Sales_98',
                                                                                                            'Annual_Profit':'Profit_98',
                                                                                                             'Sales_YOY':"YOY_98"})
new_df.head()

,Country,Sales_96,Profit_96,Sales_97,Profit_97,YOY_97,Sales_98,Profit_98,YOY_98
0,Argentina,NaN,NaN,1816.60,127.17,0.000000,6302.50,441.18,2.469393
1,Austria,25601.34,1792.10,57401.86,4018.15,1.242143,45000.65,3150.08,-0.216042
2,Belgium,6306.70,441.47,11434.48,800.42,0.813069,16083.67,1125.87,0.406594
3,Brazil,20148.82,1410.43,41941.19,2935.95,1.081571,44835.77,3138.52,0.069015
4,Canada,7372.68,516.09,31298.06,2190.90,3.245140,11525.56,806.80,-0.631748


In [32]:
new_df.fillna(0, inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    21 non-null     object 
 1   Sales_96   21 non-null     float64
 2   Profit_96  21 non-null     float64
 3   Sales_97   21 non-null     float64
 4   Profit_97  21 non-null     float64
 5   YOY_97     21 non-null     float64
 6   Sales_98   21 non-null     float64
 7   Profit_98  21 non-null     float64
 8   YOY_98     21 non-null     float64
dtypes: float64(8), object(1)
memory usage: 1.6+ KB


In [34]:
# Reorder the columns:
new_df.insert(5,'Sales_98',new_df.pop('Sales_98'))
new_df.insert(6,'Profit_98',new_df.pop('Profit_98'))
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    21 non-null     object 
 1   Sales_96   21 non-null     float64
 2   Profit_96  21 non-null     float64
 3   Sales_97   21 non-null     float64
 4   Profit_97  21 non-null     float64
 5   Sales_98   21 non-null     float64
 6   Profit_98  21 non-null     float64
 7   YOY_97     21 non-null     float64
 8   YOY_98     21 non-null     float64
dtypes: float64(8), object(1)
memory usage: 1.6+ KB


In [36]:
new_df

,Country,Sales_96,Profit_96,Sales_97,Profit_97,Sales_98,Profit_98,YOY_97,YOY_98
0,Argentina,0.00,0.00,1816.60,127.17,6302.50,441.18,0.000000,2.469393
1,Austria,25601.34,1792.10,57401.86,4018.15,45000.65,3150.08,1.242143,-0.216042
2,Belgium,6306.70,441.47,11434.48,800.42,16083.67,1125.87,0.813069,0.406594
3,Brazil,20148.82,1410.43,41941.19,2935.95,44835.77,3138.52,1.081571,0.069015
4,Canada,7372.68,516.09,31298.06,2190.90,11525.56,806.80,3.245140,-0.631748
5,Denmark,2952.40,206.66,25192.54,1763.48,4516.09,316.14,7.532902,-0.820737
6,Finland,3115.76,218.10,13437.29,940.61,2257.00,158.01,3.312685,-0.832035
7,France,17372.76,1216.09,45263.38,3168.46,18722.18,1310.61,1.605423,-0.586372
8,Germany,35407.15,2478.54,117320.18,8212.48,77557.34,5429.05,2.313460,-0.338926
9,Ireland,9123.38,638.63,20454.41,1431.85,20402.12,1428.15,1.241977,-0.002556


In [430]:
# Save the new table to SQL:
new_df.to_sql('Country_Growth', engine, index=False)

-1

<a id='specific_c'></a>
# 2- Country YOY (Totals for specific months):
_Note:_ 
Since the dataset is incomplete (the recorded sales transactions for 1996 started i July-1996 and ended in May-1998)
- The sales of 1997 (from Jul : Dec) will be compared to the sales of 1996 ***(The Second Half "H2")***
- The sales of 1997 (from Jan : Jun) will be compared to the sales of 1998 ***(The First Half "H1")***

In [432]:
# Loading Data of the second half (from Jul:Dec)for the years 1996 &1997
country_h2=pd.read_sql_query('select * from [dbo].[Country_Transactions_H2]',con_db)

In [434]:
country_h2.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit
0,Argentina,1997H2-Arg,1997,718.50,50.30
1,Austria,1996H2-Aus,1996,25601.34,1792.10
2,Austria,1997H2-Aus,1997,32697.90,2288.87
3,Belgium,1996H2-Bel,1996,6306.70,441.47
4,Belgium,1997H2-Bel,1997,4379.00,306.53


In [436]:
country_h2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        41 non-null     object 
 1   Country_ID     41 non-null     object 
 2   Year           41 non-null     int64  
 3   Annual_Sales   41 non-null     float64
 4   Annual_Profit  41 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ KB


In [438]:
#Sorting Data:
country_h2.sort_values(by= ['Country','Year'], inplace= True,ignore_index=True)
country_h2.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit
0,Argentina,1997H2-Arg,1997,718.50,50.30
1,Austria,1996H2-Aus,1996,25601.34,1792.10
2,Austria,1997H2-Aus,1997,32697.90,2288.87
3,Belgium,1996H2-Bel,1996,6306.70,441.47
4,Belgium,1997H2-Bel,1997,4379.00,306.53


In [446]:
# Calculating YOY for Sales :
country_h2['Sales_YOY'] = country_h2.groupby('Country').Annual_Sales.pct_change(periods=1)
country_h2.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997H2-Arg,1997,718.50,50.30,NaN
1,Austria,1996H2-Aus,1996,25601.34,1792.10,NaN
2,Austria,1997H2-Aus,1997,32697.90,2288.87,0.277195
3,Belgium,1996H2-Bel,1996,6306.70,441.47,NaN
4,Belgium,1997H2-Bel,1997,4379.00,306.53,-0.305659


In [448]:
country_h2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        41 non-null     object 
 1   Country_ID     41 non-null     object 
 2   Year           41 non-null     int64  
 3   Annual_Sales   41 non-null     float64
 4   Annual_Profit  41 non-null     float64
 5   Sales_YOY      20 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.1+ KB


In [450]:
country_h2

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997H2-Arg,1997,718.50,50.30,NaN
1,Austria,1996H2-Aus,1996,25601.34,1792.10,NaN
2,Austria,1997H2-Aus,1997,32697.90,2288.87,0.277195
3,Belgium,1996H2-Bel,1996,6306.70,441.47,NaN
4,Belgium,1997H2-Bel,1997,4379.00,306.53,-0.305659
5,Brazil,1996H2-Bra,1996,20148.82,1410.43,NaN
6,Brazil,1997H2-Bra,1997,27622.55,1933.64,0.370926
7,Canada,1996H2-Can,1996,7372.68,516.09,NaN
8,Canada,1997H2-Can,1997,12982.20,908.77,0.760852
9,Denmark,1996H2-Den,1996,2952.40,206.66,NaN


In [456]:
country_h2.query('Year == 1996').head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
1,Austria,1996H2-Aus,1996,25601.34,1792.10,NaN
3,Belgium,1996H2-Bel,1996,6306.70,441.47,NaN
5,Brazil,1996H2-Bra,1996,20148.82,1410.43,NaN
7,Canada,1996H2-Can,1996,7372.68,516.09,NaN
9,Denmark,1996H2-Den,1996,2952.40,206.66,NaN


In [454]:
country_h2.query('Year == 1997').shape

(21, 6)

In [458]:
# Reshaping the Data:
new_h2= country_h2.query('Year == 1996').iloc[:,[0,3,4]].rename(columns={'Annual_Sales':'Sales_96_H2',
                                                                             'Annual_Profit':'Profit_96_H2'})

new_h2=new_h2.merge(country_h2.query('Year == 1997').iloc[:,[0,3,4,5]], how = 'right').rename(columns={'Annual_Sales':'Sales_97_H2',
                                                                                                         'Annual_Profit':'Profit_97_H2',
                                                                                                         'Sales_YOY':'Sales_YOY_97_H2'})
new_h2.head()

,Country,Sales_96_H2,Profit_96_H2,Sales_97_H2,Profit_97_H2,Sales_YOY_97_H2
0,Argentina,NaN,NaN,718.50,50.30,NaN
1,Austria,25601.34,1792.10,32697.90,2288.87,0.277195
2,Belgium,6306.70,441.47,4379.00,306.53,-0.305659
3,Brazil,20148.82,1410.43,27622.55,1933.64,0.370926
4,Canada,7372.68,516.09,12982.20,908.77,0.760852


In [460]:
# Loading Data of the first half (from Jan:Jun)for the years 1997 &1998
country_h1=pd.read_sql_query('select * from [dbo].[Country_Transactions_H1]',con_db)

In [480]:
country_h1.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997H1-Arg,1997,1098.10,76.87,NaN
1,Argentina,1998H1-Arg,1998,6302.50,441.18,4.739459
2,Austria,1997H1-Aus,1997,24703.96,1729.28,NaN
3,Austria,1998H1-Aus,1998,45000.65,3150.08,0.821597
4,Belgium,1997H1-Bel,1997,7055.48,493.89,NaN


In [482]:
country_h1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        41 non-null     object 
 1   Country_ID     41 non-null     object 
 2   Year           41 non-null     int64  
 3   Annual_Sales   41 non-null     float64
 4   Annual_Profit  41 non-null     float64
 5   Sales_YOY      20 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.1+ KB


In [484]:
#Sorting Data:
country_h1.sort_values(by= ['Country','Year'], inplace= True,ignore_index=True)
country_h1.head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997H1-Arg,1997,1098.10,76.87,NaN
1,Argentina,1998H1-Arg,1998,6302.50,441.18,4.739459
2,Austria,1997H1-Aus,1997,24703.96,1729.28,NaN
3,Austria,1998H1-Aus,1998,45000.65,3150.08,0.821597
4,Belgium,1997H1-Bel,1997,7055.48,493.89,NaN


In [486]:
# Calculating YOY for Sales & Profit:
country_h1['Sales_YOY'] = country_h1.groupby('Country').Annual_Sales.pct_change(periods=1)
country_h1.head(5)

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY
0,Argentina,1997H1-Arg,1997,1098.10,76.87,NaN
1,Argentina,1998H1-Arg,1998,6302.50,441.18,4.739459
2,Austria,1997H1-Aus,1997,24703.96,1729.28,NaN
3,Austria,1998H1-Aus,1998,45000.65,3150.08,0.821597
4,Belgium,1997H1-Bel,1997,7055.48,493.89,NaN


In [488]:
country_h1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        41 non-null     object 
 1   Country_ID     41 non-null     object 
 2   Year           41 non-null     int64  
 3   Annual_Sales   41 non-null     float64
 4   Annual_Profit  41 non-null     float64
 5   Sales_YOY      20 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.1+ KB


In [111]:
country_h1.query('Year==1997').head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY,Profit_YOY
0,Argentina,1997H1-Arg,1997,1098.10,76.87,0.0,0.0
2,Austria,1997H1-Aus,1997,24703.96,1729.28,0.0,0.0
4,Belgium,1997H1-Bel,1997,7055.48,493.89,0.0,0.0
6,Brazil,1997H1-Bra,1997,14318.64,1002.31,0.0,0.0
8,Canada,1997H1-Can,1997,18315.86,1282.13,0.0,0.0


In [113]:
country_h1.query('Year==1998').head()

,Country,Country_ID,Year,Annual_Sales,Annual_Profit,Sales_YOY,Profit_YOY
1,Argentina,1998H1-Arg,1998,6302.50,441.18,4.739459,4.739300
3,Austria,1998H1-Aus,1998,45000.65,3150.08,0.821597,0.821614
5,Belgium,1998H1-Bel,1998,16083.67,1125.87,1.279600,1.279597
7,Brazil,1998H1-Bra,1998,44835.77,3138.52,2.131287,2.131287
9,Canada,1998H1-Can,1998,11525.56,806.80,-0.370733,-0.370735


In [492]:
# Reshaping the Data:
new_h1= country_h1.query('Year == 1997').iloc[:,[0,3,4]].rename(columns={'Annual_Sales':'Sales_97_H1',
                                                                             'Annual_Profit':'Profit_97_H1',})

new_h1=new_h1.merge(country_h1.query('Year == 1998').iloc[:,[0,3,4,5]], how = 'right').rename(columns={'Annual_Sales':'Sales_98_H1',
                                                                                                         'Annual_Profit':'Profit_98_H1',
                                                                                                         'Sales_YOY':'Sales_YOY_98_H1',})
new_h1.head()

,Country,Sales_97_H1,Profit_97_H1,Sales_98_H1,Profit_98_H1,Sales_YOY_98_H1
0,Argentina,1098.10,76.87,6302.50,441.18,4.739459
1,Austria,24703.96,1729.28,45000.65,3150.08,0.821597
2,Belgium,7055.48,493.89,16083.67,1125.87,1.279600
3,Brazil,14318.64,1002.31,44835.77,3138.52,2.131287
4,Canada,18315.86,1282.13,11525.56,806.80,-0.370733


In [117]:
new_h1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           21 non-null     object 
 1   Sales_97_H1       20 non-null     float64
 2   Profit_97_H1      20 non-null     float64
 3   Sales_YOY_97_H1   20 non-null     float64
 4   Profit_YOY_97_H1  20 non-null     float64
 5   Sales_98_H1       21 non-null     float64
 6   Profit_98_H1      21 non-null     float64
 7   Sales_YOY_98_H1   21 non-null     float64
 8   Profit_YOY_98_H1  21 non-null     float64
dtypes: float64(8), object(1)
memory usage: 1.6+ KB


In [125]:
new_h1

,Country,Sales_97_H1,Profit_97_H1,Sales_98_H1,Profit_98_H1,Sales_YOY_98_H1,Profit_YOY_98_H1
0,Argentina,1098.10,76.87,6302.50,441.18,4.739459,4.739300
1,Austria,24703.96,1729.28,45000.65,3150.08,0.821597,0.821614
2,Belgium,7055.48,493.89,16083.67,1125.87,1.279600,1.279597
3,Brazil,14318.64,1002.31,44835.77,3138.52,2.131287,2.131287
4,Canada,18315.86,1282.13,11525.56,806.80,-0.370733,-0.370735
5,Denmark,14541.60,1017.91,4516.09,316.14,-0.689437,-0.689422
6,Finland,9067.00,634.69,2257.00,158.01,-0.751075,-0.751044
7,France,23589.06,1651.24,18722.18,1310.61,-0.206319,-0.206287
8,Germany,53947.12,3776.33,77557.34,5429.05,0.437655,0.437652
9,Ireland,8389.93,587.31,20402.12,1428.15,1.431739,1.431680


In [494]:
new_h2.head(1)

,Country,Sales_96_H2,Profit_96_H2,Sales_97_H2,Profit_97_H2,Sales_YOY_97_H2
0,Argentina,NaN,NaN,718.5,50.3,NaN


In [496]:
new_h1.head(1)

,Country,Sales_97_H1,Profit_97_H1,Sales_98_H1,Profit_98_H1,Sales_YOY_98_H1
0,Argentina,1098.1,76.87,6302.5,441.18,4.739459


In [518]:
semi_annual_report=new_h2.merge(new_h1,how='inner')
semi_annual_report.query('Country =="Norway"').head()

,Country,Sales_96_H2,Profit_96_H2,Sales_97_H2,Profit_97_H2,Sales_YOY_97_H2,Sales_97_H1,Profit_97_H1,Sales_98_H1,Profit_98_H1,Sales_YOY_98_H1
12,Norway,1058.4,74.09,500.0,35.0,-0.527589,200.0,14.0,3976.75,278.37,18.88375


In [500]:
country_growth=semi_annual_report.iloc[:,[0,5,10]]
country_growth

,Country,Sales_YOY_97_H2,Sales_YOY_98_H1
0,Argentina,NaN,4.739459
1,Austria,0.277195,0.821597
2,Belgium,-0.305659,1.279600
3,Brazil,0.370926,2.131287
4,Canada,0.760852,-0.370733
5,Denmark,2.607553,-0.689437
6,Finland,0.402640,-0.751075
7,France,0.247604,-0.206319
8,Germany,0.789838,0.437655
9,Ireland,0.322370,1.431739


In [512]:
country_growth.iloc[:,1:3].rename(columns={'Sales_YOY_97_H2':'YOY_97','Sales_YOY_98_H1':'YOY_98'})
country_growth

,Country,YOY_97,YOY_98
0,Argentina,NaN,4.739459
1,Austria,0.277195,0.821597
2,Belgium,-0.305659,1.279600
3,Brazil,0.370926,2.131287
4,Canada,0.760852,-0.370733
5,Denmark,2.607553,-0.689437
6,Finland,0.402640,-0.751075
7,France,0.247604,-0.206319
8,Germany,0.789838,0.437655
9,Ireland,0.322370,1.431739


In [516]:
# Save the new table to SQL:
country_growth.to_sql('Actual_Country_Growth', engine, index=False)

-1

<a id='product'></a>
# 3- Products YOY

In [224]:
#Loading the Data:
products_report=pd.read_sql_query('select * from [dbo].[Products_Report]',con_db)
products_report.head()

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit
0,1996-1,1996,1,Chai,Beverages,125,14.4,194.40,1605.60,112.39
1,1996-2,1996,2,Chang,Beverages,226,15.2,417.24,3017.96,211.26
2,1996-3,1996,3,Aniseed Syrup,Condiments,30,8.0,0.00,240.00,16.80
3,1996-4,1996,4,Chef Anton's Ca,Condiments,107,17.6,31.68,1851.52,129.61
4,1996-5,1996,5,Chef Anton's Gu,Condiments,129,17.0,261.80,1931.20,135.18


In [226]:
products_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year_ID       227 non-null    object 
 1   per_Year      227 non-null    int64  
 2   Product_ID    227 non-null    int64  
 3   Product_Name  227 non-null    object 
 4   Category      227 non-null    object 
 5   Q_Sold        227 non-null    int64  
 6   Avarage_SP    227 non-null    float64
 7   Discount      227 non-null    float64
 8   N_Sales       227 non-null    float64
 9   N_Profit      227 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 17.9+ KB


In [228]:
#Sorting Values:
products_report.sort_values(by= ['Product_ID','per_Year'], inplace= True,ignore_index=True)
products_report.head(3)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit
0,1996-1,1996,1,Chai,Beverages,125,14.400,194.4,1605.6,112.39
1,1997-1,1997,1,Chai,Beverages,304,17.325,408.6,4887.0,342.10
2,1998-1,1998,1,Chai,Beverages,399,18.000,886.5,6295.5,440.69


In [230]:
# Calculating YOY:
products_report['Sales_YOY'] = products_report.groupby('Product_ID').N_Sales.pct_change(periods=1)
products_report.head(3)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit,Sales_YOY
0,1996-1,1996,1,Chai,Beverages,125,14.400,194.4,1605.6,112.39,NaN
1,1997-1,1997,1,Chai,Beverages,304,17.325,408.6,4887.0,342.10,2.043722
2,1998-1,1998,1,Chai,Beverages,399,18.000,886.5,6295.5,440.69,0.288214


In [232]:
products_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year_ID       227 non-null    object 
 1   per_Year      227 non-null    int64  
 2   Product_ID    227 non-null    int64  
 3   Product_Name  227 non-null    object 
 4   Category      227 non-null    object 
 5   Q_Sold        227 non-null    int64  
 6   Avarage_SP    227 non-null    float64
 7   Discount      227 non-null    float64
 8   N_Sales       227 non-null    float64
 9   N_Profit      227 non-null    float64
 10  Sales_YOY     150 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 19.6+ KB


In [234]:
products_report

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit,Sales_YOY
0,1996-1,1996,1,Chai,Beverages,125,14.4000,194.40,1605.60,112.39,NaN
1,1997-1,1997,1,Chai,Beverages,304,17.3250,408.60,4887.00,342.10,2.043722
2,1998-1,1998,1,Chai,Beverages,399,18.0000,886.50,6295.50,440.69,0.288214
3,1996-2,1996,2,Chang,Beverages,226,15.2000,417.24,3017.96,211.26,NaN
4,1997-2,1997,2,Chang,Beverages,435,17.9444,561.45,7038.55,492.71,1.332221
...,...,...,...,...,...,...,...,...,...,...,...
222,1997-76,1997,76,Lakkalikööri,Beverages,447,17.2421,328.50,7379.10,516.54,2.606175
223,1998-76,1998,76,Lakkalikööri,Beverages,388,18.0000,648.90,6335.10,443.46,-0.141481
224,1996-77,1996,77,Original Frankf,Condiments,63,10.4000,0.00,655.20,45.87,NaN
225,1997-77,1997,77,Original Frankf,Condiments,432,11.9294,420.42,4761.38,333.32,6.267063


In [236]:
products_report.fillna(0,inplace=True)
products_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year_ID       227 non-null    object 
 1   per_Year      227 non-null    int64  
 2   Product_ID    227 non-null    int64  
 3   Product_Name  227 non-null    object 
 4   Category      227 non-null    object 
 5   Q_Sold        227 non-null    int64  
 6   Avarage_SP    227 non-null    float64
 7   Discount      227 non-null    float64
 8   N_Sales       227 non-null    float64
 9   N_Profit      227 non-null    float64
 10  Sales_YOY     227 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 19.6+ KB


##### Reshaping the Data

In [239]:
products_report.query('per_Year == 1996').head()

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit,Sales_YOY
0,1996-1,1996,1,Chai,Beverages,125,14.4,194.40,1605.60,112.39,0.0
3,1996-2,1996,2,Chang,Beverages,226,15.2,417.24,3017.96,211.26,0.0
6,1996-3,1996,3,Aniseed Syrup,Condiments,30,8.0,0.00,240.00,16.80,0.0
9,1996-4,1996,4,Chef Anton's Ca,Condiments,107,17.6,31.68,1851.52,129.61,0.0
12,1996-5,1996,5,Chef Anton's Gu,Condiments,129,17.0,261.80,1931.20,135.18,0.0


In [241]:
products_report.query('per_Year == 1997').shape

(77, 11)

In [243]:
products_report.query('per_Year == 1998').shape

(76, 11)

In [245]:
yoy= products_report.query('per_Year == 1997').iloc[:,[2,3,4,8,9,10]].rename(columns={'N_Sales':'N_Sales_97','N_Profit':'N_Profit_97','Sales_YOY':'YOY_97'}) 
yoy.head(1)

,Product_ID,Product_Name,Category,N_Sales_97,N_Profit_97,YOY_97
1,1,Chai,Beverages,4887.0,342.1,2.043722


In [247]:
_1996=products_report.query('per_Year == 1996')
_1996.head(1)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit,Sales_YOY
0,1996-1,1996,1,Chai,Beverages,125,14.4,194.4,1605.6,112.39,0.0


In [190]:
# Reshaping the Data:
yoy=yoy.merge(_1996.iloc[:,[2,3,4,8,9]], how = 'left').rename(columns={'N_Sales':'N_Sales_96','N_Profit':'N_Profit_96'})
yoy.head(1)

,Product_ID,Product_Name,Category,N_Sales_97,N_Profit_97,YOY_97,N_Sales_96,N_Profit_96
0,1,Chai,Beverages,4887.0,342.1,2.043722,1605.6,112.39


In [192]:
_1998=products_report.query('per_Year == 1998')
_1998.head(1)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Avarage_SP,Discount,N_Sales,N_Profit,Sales_YOY
2,1998-1,1998,1,Chai,Beverages,399,18.0,886.5,6295.5,440.69,0.288214


In [194]:
yoy=yoy.merge(_1998.iloc[:,[2,3,4,8,9,10]], how = 'left').rename(columns={'N_Sales':'N_Sales_98','N_Profit':'N_Profit_98','Sales_YOY':'YOY_98'})
yoy.head()

,Product_ID,Product_Name,Category,N_Sales_97,N_Profit_97,YOY_97,N_Sales_96,N_Profit_96,N_Sales_98,N_Profit_98,YOY_98
0,1,Chai,Beverages,4887.00,342.10,2.043722,1605.60,112.39,6295.50,440.69,0.288214
1,2,Chang,Beverages,7038.55,492.71,1.332221,3017.96,211.26,6299.45,440.98,-0.105007
2,3,Aniseed Syrup,Condiments,1724.00,120.68,6.183333,240.00,16.80,1080.00,75.60,-0.373550
3,4,Chef Anton's Ca,Condiments,5214.88,365.04,1.816540,1851.52,129.61,1501.50,105.11,-0.712074
4,5,Chef Anton's Gu,Condiments,373.63,26.16,-0.806530,1931.20,135.18,3042.38,212.98,7.142762


In [196]:
yoy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product_ID    77 non-null     int64  
 1   Product_Name  77 non-null     object 
 2   Category      77 non-null     object 
 3   N_Sales_97    77 non-null     float64
 4   N_Profit_97   77 non-null     float64
 5   YOY_97        77 non-null     float64
 6   N_Sales_96    74 non-null     float64
 7   N_Profit_96   74 non-null     float64
 8   N_Sales_98    76 non-null     float64
 9   N_Profit_98   76 non-null     float64
 10  YOY_98        76 non-null     float64
dtypes: float64(8), int64(1), object(2)
memory usage: 7.2+ KB


In [198]:
# Replace the Nill Values with 0
yoy.fillna(0,inplace=True)
yoy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product_ID    77 non-null     int64  
 1   Product_Name  77 non-null     object 
 2   Category      77 non-null     object 
 3   N_Sales_97    77 non-null     float64
 4   N_Profit_97   77 non-null     float64
 5   YOY_97        77 non-null     float64
 6   N_Sales_96    77 non-null     float64
 7   N_Profit_96   77 non-null     float64
 8   N_Sales_98    77 non-null     float64
 9   N_Profit_98   77 non-null     float64
 10  YOY_98        77 non-null     float64
dtypes: float64(8), int64(1), object(2)
memory usage: 7.2+ KB


In [200]:
yoy.insert(3,'N_Sales_96',yoy.pop('N_Sales_96'))
yoy.insert(5,'N_Sales_98',yoy.pop('N_Sales_98'))
yoy.insert(6,'N_Profit_96',yoy.pop('N_Profit_96'))
yoy.insert(7,'N_Profit_98',yoy.pop('N_Profit_98'))
yoy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product_ID    77 non-null     int64  
 1   Product_Name  77 non-null     object 
 2   Category      77 non-null     object 
 3   N_Sales_96    77 non-null     float64
 4   N_Sales_97    77 non-null     float64
 5   N_Sales_98    77 non-null     float64
 6   N_Profit_96   77 non-null     float64
 7   N_Profit_98   77 non-null     float64
 8   N_Profit_97   77 non-null     float64
 9   YOY_97        77 non-null     float64
 10  YOY_98        77 non-null     float64
dtypes: float64(8), int64(1), object(2)
memory usage: 7.2+ KB


In [202]:
yoy

,Product_ID,Product_Name,Category,N_Sales_96,N_Sales_97,N_Sales_98,N_Profit_96,N_Profit_98,N_Profit_97,YOY_97,YOY_98
0,1,Chai,Beverages,1605.60,4887.00,6295.50,112.39,440.69,342.10,2.043722,0.288214
1,2,Chang,Beverages,3017.96,7038.55,6299.45,211.26,440.98,492.71,1.332221,-0.105007
2,3,Aniseed Syrup,Condiments,240.00,1724.00,1080.00,16.80,75.60,120.68,6.183333,-0.373550
3,4,Chef Anton's Ca,Condiments,1851.52,5214.88,1501.50,129.61,105.11,365.04,1.816540,-0.712074
4,5,Chef Anton's Gu,Condiments,1931.20,373.63,3042.38,135.18,212.98,26.16,-0.806530,7.142762
...,...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,Seafood,300.00,3517.50,179.70,21.00,12.58,246.23,10.725000,-0.948913
73,74,Longlife Tofu,Produce,1032.00,1000.50,400.00,72.24,28.00,70.04,-0.030523,-0.600200
74,75,Rhönbräu Kloste,Beverages,738.42,4485.55,2953.53,51.68,206.78,314.02,5.074524,-0.341546
75,76,Lakkalikööri,Beverages,2046.24,7379.10,6335.10,143.24,443.46,516.54,2.606175,-0.141481


In [208]:
# Save the new table to SQL:
yoy.to_sql('Product_Growth', engine, index=False)

-1

<a id='specific_p_h2'></a>
# 4- Products YOY (Totals for specific months) 
_Note:_ 
Since the dataset is incomplete (the recorded sales transactions for 1996 started i July-1996 and ended in May-1998)
- The sales of 1997 (from Jul : Dec) will be compared to the sales of 1996 ***(The Second Half "H2")***
- The sales of 1997 (from Jan : Jun) will be compared to the sales of 1998 ***(The First Half "H1")***

In [647]:
# Loading Data of the second half (from Jul:Dec)for the years 1996 &1997
product_h2=pd.read_sql_query('select * from [dbo].[P_Sales_H2]',con_db)
product_h2.head()

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight
0,1996-10IkH2,1996,10,Ikura,Seafood,85,4,12.6936,24.80,2108.0,178.56,0.25,0.0098,0.0847,1929.44,0.0092,482.360,135.06,16.35
1,1996-11QuH2,1996,11,Queso Cabrales,Dairy Products,110,5,10.5663,16.24,1814.4,208.32,0.40,0.0114,0.1148,1606.08,0.0077,321.216,112.42,42.95
2,1996-12QuH2,1996,12,Queso Manchego,Dairy Products,12,1,2.2800,30.40,364.8,18.24,0.05,0.0010,0.0500,346.56,0.0016,346.560,24.26,8.58
3,1996-13KoH2,1996,13,Konbu,Seafood,90,6,2.7789,4.80,432.0,9.60,0.10,0.0005,0.0222,422.40,0.0020,70.400,29.57,17.46
4,1996-14ToH2,1996,14,Tofu,Produce,85,5,9.1164,18.60,1581.0,195.30,0.25,0.0107,0.1235,1385.70,0.0066,277.140,97.00,17.57


In [648]:
product_h2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year_ID                150 non-null    object 
 1   per_Year               150 non-null    int64  
 2   Product_ID             150 non-null    int64  
 3   Product_Name           150 non-null    object 
 4   Category               150 non-null    object 
 5   Q_Sold                 150 non-null    int64  
 6   Assigned_Orders        150 non-null    int64  
 7   SP_Order               150 non-null    float64
 8   SP                     150 non-null    float64
 9   Total_Sales            150 non-null    float64
 10  Discount               150 non-null    float64
 11  Discount_pct           150 non-null    float64
 12  Discount_WA            150 non-null    float64
 13  Deducted_DIscount_pct  150 non-null    float64
 14  N_Sales                150 non-null    float64
 15  NS_pct

In [649]:
#Sorting Values:
product_h2.sort_values(by= ['Product_ID','per_Year'], inplace= True,ignore_index=True)
product_h2.head(2)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight
0,1996-1ChH2,1996,1,Chai,Beverages,125,6,10.5631,14.4,1800.0,194.4,0.5,0.0106,0.1080,1605.6,0.0077,267.6,112.39,39.17
1,1997-1ChH2,1997,1,Chai,Beverages,197,10,21.7302,18.0,3546.0,243.0,0.7,0.0133,0.0685,3303.0,0.0158,330.3,231.22,96.25


In [650]:
# Calculating YOY:
product_h2['YOY'] = product_h2.groupby('Product_ID').N_Sales.pct_change(periods=1) 
product_h2.head(2)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight,YOY
0,1996-1ChH2,1996,1,Chai,Beverages,125,6,10.5631,14.4,1800.0,194.4,0.5,0.0106,0.1080,1605.6,0.0077,267.6,112.39,39.17,NaN
1,1997-1ChH2,1997,1,Chai,Beverages,197,10,21.7302,18.0,3546.0,243.0,0.7,0.0133,0.0685,3303.0,0.0158,330.3,231.22,96.25,1.057175


In [651]:
product_h2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year_ID                150 non-null    object 
 1   per_Year               150 non-null    int64  
 2   Product_ID             150 non-null    int64  
 3   Product_Name           150 non-null    object 
 4   Category               150 non-null    object 
 5   Q_Sold                 150 non-null    int64  
 6   Assigned_Orders        150 non-null    int64  
 7   SP_Order               150 non-null    float64
 8   SP                     150 non-null    float64
 9   Total_Sales            150 non-null    float64
 10  Discount               150 non-null    float64
 11  Discount_pct           150 non-null    float64
 12  Discount_WA            150 non-null    float64
 13  Deducted_DIscount_pct  150 non-null    float64
 14  N_Sales                150 non-null    float64
 15  NS_pct

#### Reshaping H2 Data

In [653]:
product_96_h2= product_h2.query('per_Year== 1996')
product_96_h2.shape

(74, 20)

In [654]:
product_97_h2= product_h2.query('per_Year== 1997')
product_97_h2.shape

(76, 20)

In [655]:
# Reshaping the Data:
new_p_h2= product_96_h2.iloc[:,[2,3,4,14,17]].rename(columns={'N_Sales':'N_Sales_96_H2',
                                                            'Net_Profit':'Profit_96_H2'})
new_p_h2.head(1)

,Product_ID,Product_Name,Category,N_Sales_96_H2,Profit_96_H2
0,1,Chai,Beverages,1605.6,112.39


In [661]:
new_p_h2=new_p_h2.merge(product_97_h2.iloc[:,[2,3,4,14,17,19]], how = 'right').rename(columns={'N_Sales':'N_Sales_97_H2',
                                                                                                 'Net_Profit':'Profit_97_H2',
                                                                                                   'YOY':'YOY_97'})
new_p_h2.head()

,Product_ID,Product_Name,Category,N_Sales_96_H2,Profit_96_H2,N_Sales_97_H2,Profit_97_H2,YOY_97
0,1,Chai,Beverages,1605.60,112.39,3303.00,231.22,1.057175
1,2,Chang,Beverages,3017.96,211.26,4374.75,306.24,0.449572
2,3,Aniseed Syrup,Condiments,240.00,16.80,580.00,40.60,1.416667
3,4,Chef Anton's Ca,Condiments,1851.52,129.61,2019.60,141.37,0.090779
4,5,Chef Anton's Gu,Condiments,1931.20,135.18,373.63,26.16,-0.806530


In [667]:
# Loading Data of the first half (from Jan:Jun)for the years 1997 &1998
product_h1=pd.read_sql_query('select * from [dbo].[P_Sales_H1]',con_db)
product_h1.head()

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight
0,1997-10IkH1,1997,10,Ikura,Seafood,77,6,10.2886,27.9000,2052.2,148.80,0.3,0.0082,0.0725,1903.40,0.0067,317.2333,133.23,18.52
1,1997-11QuH1,1997,11,Queso Cabrales,Dairy Products,245,14,23.7118,18.9000,4716.6,329.91,0.8,0.0183,0.0699,4386.69,0.0155,313.3350,307.07,71.84
2,1997-12QuH1,1997,12,Queso Manchego,Dairy Products,60,3,10.0135,35.4666,2166.0,313.50,0.4,0.0173,0.1447,1852.50,0.0065,617.5000,129.68,8.06
3,1997-13KoH1,1997,13,Konbu,Seafood,31,5,0.9807,5.5200,182.4,0.96,0.1,0.0000,0.0052,181.44,0.0006,36.2880,12.70,6.42
4,1997-14ToH1,1997,14,Tofu,Produce,205,8,22.5210,20.9250,4398.9,232.50,0.5,0.0128,0.0528,4166.40,0.0148,520.8000,291.65,53.61


In [668]:
product_h1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year_ID                151 non-null    object 
 1   per_Year               151 non-null    int64  
 2   Product_ID             151 non-null    int64  
 3   Product_Name           151 non-null    object 
 4   Category               151 non-null    object 
 5   Q_Sold                 151 non-null    int64  
 6   Assigned_Orders        151 non-null    int64  
 7   SP_Order               151 non-null    float64
 8   SP                     151 non-null    float64
 9   Total_Sales            151 non-null    float64
 10  Discount               151 non-null    float64
 11  Discount_pct           151 non-null    float64
 12  Discount_WA            151 non-null    float64
 13  Deducted_DIscount_pct  151 non-null    float64
 14  N_Sales                151 non-null    float64
 15  NS_pct

In [669]:
#Sorting Values:
product_h1.sort_values(by= ['Product_ID','per_Year'], inplace= True,ignore_index=True)
product_h1.head(2)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight
0,1997-1ChH1,1997,1,Chai,Beverages,107,6,8.5621,16.2,1749.6,165.6,0.35,0.0091,0.0946,1584.0,0.0056,264.0000,110.88,25.02
1,1998-1ChH1,1998,1,Chai,Beverages,399,16,23.3166,18.0,7182.0,886.5,1.40,0.0304,0.1234,6295.5,0.0142,393.4687,440.69,151.07


In [670]:
# Calculating YOY:
product_h1['YOY'] = product_h1.groupby('Product_ID').N_Sales.pct_change(periods=1) 
product_h1.head(2)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight,YOY
0,1997-1ChH1,1997,1,Chai,Beverages,107,6,8.5621,16.2,1749.6,165.6,0.35,0.0091,0.0946,1584.0,0.0056,264.0000,110.88,25.02,NaN
1,1998-1ChH1,1998,1,Chai,Beverages,399,16,23.3166,18.0,7182.0,886.5,1.40,0.0304,0.1234,6295.5,0.0142,393.4687,440.69,151.07,2.974432


In [675]:
product_h1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year_ID                151 non-null    object 
 1   per_Year               151 non-null    int64  
 2   Product_ID             151 non-null    int64  
 3   Product_Name           151 non-null    object 
 4   Category               151 non-null    object 
 5   Q_Sold                 151 non-null    int64  
 6   Assigned_Orders        151 non-null    int64  
 7   SP_Order               151 non-null    float64
 8   SP                     151 non-null    float64
 9   Total_Sales            151 non-null    float64
 10  Discount               151 non-null    float64
 11  Discount_pct           151 non-null    float64
 12  Discount_WA            151 non-null    float64
 13  Deducted_DIscount_pct  151 non-null    float64
 14  N_Sales                151 non-null    float64
 15  NS_pct

#### Reshaping H1 Data

In [678]:
product_h1_97=product_h1.query('per_Year == 1997')
product_h1_97.head(1)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight,YOY
0,1997-1ChH1,1997,1,Chai,Beverages,107,6,8.5621,16.2,1749.6,165.6,0.35,0.0091,0.0946,1584.0,0.0056,264.0,110.88,25.02,NaN


In [680]:
new_p_h1=product_h1_97.iloc[:,[2,3,4,14,17]].rename(columns={'N_Sales':'N_Sales_97_H1','Net_Profit':'Net_Profit_97_H1'})
new_p_h1.head(1)

,Product_ID,Product_Name,Category,N_Sales_97_H1,Net_Profit_97_H1
0,1,Chai,Beverages,1584.0,110.88


In [682]:
product_h1_98=product_h1.query('per_Year == 1998')
product_h1_98.head(1)

,Year_ID,per_Year,Product_ID,Product_Name,Category,Q_Sold,Assigned_Orders,SP_Order,SP,Total_Sales,Discount,Discount_pct,Discount_WA,Deducted_DIscount_pct,N_Sales,NS_pct,Avg_N_Sales,Net_Profit,Freight,YOY
1,1998-1ChH1,1998,1,Chai,Beverages,399,16,23.3166,18.0,7182.0,886.5,1.4,0.0304,0.1234,6295.5,0.0142,393.4687,440.69,151.07,2.974432


In [684]:
new_p_h1=new_p_h1.merge(product_h1_98.iloc[:,[2,3,4,14,17,19]],how='right').rename(columns={'N_Sales':'N_Sales_98_H1','Net_Profit':'Net_Profit_98_H1',
                                                                                           'YOY':'YOY_98'})


In [686]:
new_p_h1.head(1)

,Product_ID,Product_Name,Category,N_Sales_97_H1,Net_Profit_97_H1,N_Sales_98_H1,Net_Profit_98_H1,YOY_98
0,1,Chai,Beverages,1584.0,110.88,6295.5,440.69,2.974432


 #### Merging H1 & H2

In [689]:
new_p_h2.shape

(76, 8)

In [691]:
new_p_h1.shape

(76, 8)

In [693]:
actual_product_growth= new_p_h2.iloc[:,[0,1,2,7]]
actual_product_growth.head(1)

,Product_ID,Product_Name,Category,YOY_97
0,1,Chai,Beverages,1.057175


In [695]:
actual_product_growth=actual_product_growth.merge(new_p_h1.iloc[:,[0,1,2,7]], how = 'right')

In [697]:
actual_product_growth

,Product_ID,Product_Name,Category,YOY_97,YOY_98
0,1,Chai,Beverages,1.057175,2.974432
1,2,Chang,Beverages,0.449572,1.364836
2,3,Aniseed Syrup,Condiments,1.416667,-0.055944
3,4,Chef Anton's Ca,Condiments,0.090779,-0.530088
4,5,Chef Anton's Gu,Condiments,-0.806530,NaN
...,...,...,...,...,...
71,73,Röd Kaviar,Seafood,7.625000,-0.806774
72,74,Longlife Tofu,Produce,-0.503391,-0.180328
73,75,Rhönbräu Kloste,Beverages,2.100336,0.344837
74,76,Lakkalikööri,Beverages,1.040376,0.977247


In [699]:
# Save the new table to SQL:
actual_product_growth.to_sql('Actual_Product_Growth', engine, index=False)

-1